## Imports 

In [1]:
import math
import numpy as np
from sklearn import model_selection
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from random import *
import warnings
warnings.filterwarnings("ignore")

## Simulated Annealing

In [2]:
# Simulated Annealing function steps
# @parameters 
# sol - random solution (ML model)
# X_train, y_train - training data
# X_test, y_test - testing data
# @return solution and cost

# 1. Generate a random solution
# 2. Calculate its cost using a cost function (accuracy of the ML Model)
# 3. Generate a random neigboring solution
# 4. Calculate new solutions cost (accuracy of the ML model)
# 5. Compare solutions
#     - If c_new > c_old move to the new solution
#     - If c_new < c_old maybe move to the new solution
# 6. Repeat steps until an acceptable solution is found or max number of iterations is reached
def sa(sol, X_train, y_train, X_test, y_test):
    t = 1.0
    t_min = 0.00001
    alpha = 0.9
    old_cost = cost(sol, X_train, y_train, X_test, y_test)
    while t > t_min:
        i = 1
        while i <= 100:
            new_sol = neighbor(sol)
            new_cost = cost(new_sol, X_train, y_train, X_test, y_test)
            ap = acceptance_probability(old_cost, new_cost, t)
            rnd = random()
            print(ap, rnd)
            if ap > rnd:
                sol = new_sol
                old_cost = new_cost
            i += 1
        t = t * alpha
    return sol, old_cost          

In [3]:
# Function to generate neighboring solution
# Solution is defined as a machine learning model along with a set of parameters
# i.e.,
# solution = {
#     model: 'LinearRegression',
#     parameters: {
#         fit_intercept: true,
#         normalize: true,
#         copy_X: false,
#         n_jobs: 4,
#     }
# }
def neighbor(sol):
    # Use the search_space to find a new neighbor of the current solution and return that
    
    # Chose a random parameter and modify it 
    parameter = choice(list(search_space[sol['name']].keys())) # random.choice()
    # print('Selected parameter ', parameter)
    
    parameter_space = search_space[sol['name']][parameter]
    # print('Parameter space', parameter_space)
    
    # Grab the random parameter from our current solution and change it
    current_parameter_val = sol['parameters'][parameter]
    # print('Current parameter value', current_parameter_val)
    
    # Grab the current index of the selected parameter of our model
    current_index = parameter_space.index(current_parameter_val)
    
    # In one step modify the value of the selected parameter
    if current_index == 0:
        # index = 0 -> index++
        modified_parameter = parameter_space[1];
    elif current_index == len(parameter_space):
        # index = length -> index--
        modified_parameter = parameter_space[current_index - 1]
    else: 
        # index = index + random(-1,1)
        modified_parameter = parameter_space[(current_index + choice([-1, 1])) % len(parameter_space)]
    
    # Create a new solution copy the current one and replace the randomly chosen parameter
    new_sol = sol
    new_sol['parameters'][parameter] = modified_parameter
    
    # print('New parameter value', modified_parameter)
    
    # Return new neighboring solution
    return new_sol

In [4]:
# Function to return the cost of a current solution
# In our case the cost is the accuracy (or we can use other metrics) of the current ML model / parameter configuration
# TODO: RMSE or some other metric?
def cost(sol, X_train, y_train, X_test, y_test): 
    # Get model
    model = get_model(sol['name'], sol['parameters'])
    
    # Train model on data
    model.fit(X_train, y_train)
    
    # Predictions
    predictions = model.predict(X_test)
    
    # Get accuracy or some other metric and return
    score = model.score(X_test, y_test)
    print('Score: ', score)
    return score

In [5]:
# Function to get a model with a parameter configuration
def get_model(name, parameters):
    # LogisticRegression
    if name == 'LogisticRegression':
        lr = LogisticRegression(**parameters)
        return lr
    # Model2
    elif name == '':
        return
    # Model3
    elif name == '':
        return
    # Model4
    elif name == '':
        return
    # Model5
    elif name == '':
        return
    # Default
    else:
        return

In [6]:
# Function which recommends if we should jump to a new solutions or not
# 1.0 - definitely switch
# 0.0 - definitely stay put
# 0.5 - 50/50 odds of switching
# Usually calculated by e^((c_new - c_old)/t)

def acceptance_probability(old_cost, new_cost, t): 
    if new_cost > old_cost:
        return 1.0
    else:
        return np.exp((new_cost - old_cost)/t)

## ML Models

In [7]:
# TODO: Define 5 ML models

# Search Space that will be used to define our neighborhood of ML models and hyperparameters
# Basically our dictionary defining the model, its most important parameters, and their value ranges
search_space = {
    'LogisticRegression': {
        'penalty': ['l1', 'l2'],
        'C': np.logspace(-4, 4, 20).tolist(),
        'solver': ['liblinear', 'saga'],
    },
    'Model2': {
        'property1': 'value_range',
        'property2': 'value_range',
        'property3': 'value_range',
        'property4': 'value_range'
    },
    'Model3': {
        'property1': 'value_range',
        'property2': 'value_range',
        'property3': 'value_range',
        'property4': 'value_range'
    },
    'Model4': {
        'property1': 'value_range',
        'property2': 'value_range',
        'property3': 'value_range',
        'property4': 'value_range'
    },
    'Model5': {
        'property1': 'value_range',
        'property2': 'value_range',
        'property3': 'value_range',
        'property4': 'value_range'
    }
}

## Datasets

In [8]:
# TODO: Define datasets

# Iris
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

## Evaluation

In [9]:
# Some random solution that we will pass to the sa() to start with

# LogisticRegression
# TODO: Make this an array with our 5 ML models and their parameter value ranges
# iterate over the models and call the simulated annealing function to tune the paramters
# record the solution and scores and report the best performing combination
random_lr_solution = {
    'name': 'LogisticRegression',
    'parameters': {
        'penalty': 'l2',
        'C': 0.23357214690901212,
        'solver': 'liblinear',
        'n_jobs': -1
    }
}

solution, score = sa(random_lr_solution, X_train, y_train, X_test, y_test)

Score:  0.9111111111111111
Score:  0.8888888888888888
0.9780228724846005 0.7464754876853152
Score:  1.0
1.0 0.9598637049536676
Score:  0.9777777777777777
0.9780228724846005 0.36074091129144037
Score:  1.0
1.0 0.458415416715673
Score:  1.0
1.0 0.8036778332969132
Score:  1.0
1.0 0.9131863287058523
Score:  1.0
1.0 0.4680351028711145
Score:  1.0
1.0 0.3259474305285377
Score:  1.0
1.0 0.5179326398288321
Score:  1.0
1.0 0.48692642295047284
Score:  1.0
1.0 0.9882672438534446
Score:  1.0
1.0 0.15771338227698695
Score:  1.0
1.0 0.4002211410133455
Score:  1.0
1.0 0.15311641102553997
Score:  1.0
1.0 0.78667920344404
Score:  1.0
1.0 0.6275757978756585
Score:  1.0
1.0 0.964552513530164
Score:  1.0
1.0 0.6238637660372515
Score:  1.0
1.0 0.2327124705860083
Score:  1.0
1.0 0.9104382578280689
Score:  1.0
1.0 0.07789153844890628
Score:  1.0
1.0 0.32778075556826247
Score:  1.0
1.0 0.3344618309710292
Score:  1.0
1.0 0.29098634188671646
Score:  1.0
1.0 0.9600516392555077
Score:  1.0
1.0 0.7966810565266982


Score:  0.9333333333333333
1.0 0.47518857253923175
Score:  0.9555555555555556
1.0 0.7053805758359571
Score:  0.9777777777777777
1.0 0.7292679222033116
Score:  0.8444444444444444
0.8623033568332588 0.44735237298957453
Score:  0.8888888888888888
1.0 0.9048982969249457
Score:  0.9111111111111111
1.0 0.1345135509493911
Score:  0.9333333333333333
1.0 0.6302213313301467
Score:  0.9111111111111111
0.9756109800648459 0.7039711373641522
Score:  0.9333333333333333
1.0 0.9431205716032572
Score:  0.9555555555555556
1.0 0.7197338882001877
Score:  0.9333333333333333
0.9756109800648459 0.49302522896176193
Score:  0.9111111111111111
0.9756109800648459 0.16720310886184309
Score:  0.8222222222222222
0.9059551911095096 0.12194215645197337
Score:  0.8222222222222222
1.0 0.8113589038681155
Score:  0.7777777777777778
0.9518167844230894 0.47011643283848037
Score:  0.8444444444444444
1.0 0.19385748307820339
Score:  0.7777777777777778
0.9286029058931804 0.3798600426162091
Score:  0.7555555555555555
0.975610980

Score:  1.0
1.0 0.9407306745292183
Score:  0.28888888888888886
0.3770184861839602 0.8157734595991141
Score:  0.4222222222222222
0.45268348358964605 0.3496965938722185
Score:  0.4222222222222222
1.0 0.4475253354101545
Score:  0.4222222222222222
1.0 0.29455236784046923
Score:  0.28888888888888886
0.8328523126011032 0.0453020302653645
Score:  0.4222222222222222
1.0 0.9173547884211861
Score:  0.28888888888888886
0.8328523126011032 0.23627494613461264
Score:  0.4222222222222222
1.0 0.534713560114578
Score:  0.28888888888888886
0.8328523126011032 0.3021160620492974
Score:  0.28888888888888886
1.0 0.862939351325061
Score:  0.4222222222222222
1.0 0.9054818870673585
Score:  0.4222222222222222
1.0 0.10260036900236003
Score:  0.4222222222222222
1.0 0.9209005378493753
Score:  0.28888888888888886
0.8160978200413057 0.8899176330464704
Score:  0.28888888888888886
0.8160978200413057 0.5651719502765513
Score:  0.35555555555555557
1.0 0.9036405203696477
Score:  0.35555555555555557
1.0 0.6055806957270893

Score:  0.28888888888888886
1.0 0.043939101268718805
Score:  0.28888888888888886
1.0 0.8205379004515727
Score:  0.28888888888888886
1.0 0.9496327689398085
Score:  0.28888888888888886
1.0 0.9070534025707875
Score:  0.28888888888888886
1.0 0.6070249324206064
Score:  0.28888888888888886
1.0 0.3652495328078398
Score:  0.4222222222222222
1.0 0.1252324702710491
Score:  0.4222222222222222
1.0 0.44291412543413056
Score:  0.4222222222222222
1.0 0.1867490953780575
Score:  0.4222222222222222
1.0 0.43945634170793757
Score:  0.28888888888888886
0.7978767271035134 0.025986269107281745
Score:  0.7777777777777778
1.0 0.07152706927840768
Score:  0.28888888888888886
0.43694951093029166 0.6266990694414291
Score:  0.7777777777777778
1.0 0.2135930040975308
Score:  0.7777777777777778
1.0 0.8768830625838594
Score:  0.7777777777777778
1.0 0.9282763459721478
Score:  0.3111111111111111
0.45370680309443345 0.1892032512759998
Score:  0.7777777777777778
1.0 0.9013551493632667
Score:  0.3111111111111111
0.453706803

Score:  0.28888888888888886
1.0 0.3676550859603017
Score:  0.7777777777777778
1.0 0.4351852933329712
Score:  0.7111111111111111
0.882104195136576 0.18713524182353092
Score:  0.7111111111111111
1.0 0.8644265255345503
Score:  0.4222222222222222
0.5806567948963897 0.15763365097989845
Score:  0.4222222222222222
1.0 0.38584820627066463
Score:  0.4222222222222222
1.0 0.1415441584780417
Score:  0.4222222222222222
1.0 0.33576341551608524
Score:  0.4222222222222222
1.0 0.7330506673084103
Score:  0.28888888888888886
0.7781078110775463 0.032654859560880545
Score:  0.3333333333333333
1.0 0.23528128883875
Score:  0.28888888888888886
0.9197714504257793 0.23574021948018375
Score:  0.28888888888888886
1.0 0.9463478631287355
Score:  0.4222222222222222
1.0 0.19195080384139696
Score:  0.4222222222222222
1.0 0.08566162444136904
Score:  0.4222222222222222
1.0 0.20429492424118723
Score:  0.28888888888888886
0.7567162902439261 0.7299170477605796
Score:  0.7777777777777778
1.0 0.22282486466966334
Score:  0.31

...

Score:  0.7777777777777778
0.0 0.8298643391089802
Score:  0.28888888888888886
0.0 0.0720162498894501
Score:  0.7777777777777778
0.0 0.5449080853160252
Score:  0.7777777777777778
0.0 0.6459518341292185
Score:  0.8222222222222222
0.0 0.4691996381026351
Score:  0.7777777777777778
0.0 0.1720755991661096
Score:  0.8222222222222222
0.0 0.22142870775037293
Score:  0.7777777777777778
0.0 0.48005808572133757
Score:  0.8444444444444444
0.0 0.20848962380212488
Score:  0.9777777777777777
0.0 0.7992250776875132
Score:  1.0
1.0 0.19938180086678248
Score:  0.9777777777777777
0.0 0.24825811988217783
Score:  1.0
1.0 0.8866230529993628
Score:  0.9777777777777777
0.0 0.3029786180562195
Score:  1.0
1.0 0.20577194879194682
Score:  0.9555555555555556
0.0 0.8349866786369733
Score:  1.0
1.0 0.7435260743083634
Score:  1.0
1.0 0.30217937165884867
Score:  1.0
1.0 0.5935963808964194
Score:  1.0
1.0 0.7422054235813218
Score:  1.0
1.0 0.8065795269550171
Score:  1.0
1.0 0.10085783986977515
Score:  1.0
1.0 0.46609893

Score:  1.0
1.0 0.9494821672978414
Score:  1.0
1.0 0.6232171690480862
Score:  1.0
1.0 0.4200088165116863
Score:  1.0
1.0 0.08796413629839606
Score:  1.0
1.0 0.42351870268636616
Score:  1.0
1.0 0.9767245403435308
Score:  1.0
1.0 0.1538006226077644
Score:  1.0
1.0 0.42720212440982286
Score:  1.0
1.0 0.8637627318718208
Score:  1.0
1.0 0.3336787034672829
Score:  1.0
1.0 0.22838756751227307
Score:  1.0
1.0 0.30018736660095535
Score:  1.0
1.0 0.8361068568100135
Score:  1.0
1.0 0.2492416485339991
Score:  1.0
1.0 0.2303724259605998
Score:  1.0
1.0 0.5495464786142095
Score:  1.0
1.0 0.5551219820560132
Score:  1.0
1.0 0.4749019691238924
Score:  1.0
1.0 0.9405906141246038
Score:  1.0
1.0 0.930361366477241
Score:  1.0
1.0 0.37520150659946283
Score:  1.0
1.0 0.2798359490549457
Score:  1.0
1.0 0.09136629083628856
Score:  1.0
1.0 0.0894246670973291
Score:  1.0
1.0 0.6161141867510701
Score:  1.0
1.0 0.6634471200062649
Score:  1.0
1.0 0.5269067813725238
Score:  1.0
1.0 0.23033464319427144
Score:  1.0
1

Score:  0.28888888888888886
0.0 0.6429990836574431
Score:  0.4222222222222222
0.0 0.23293576538365846
Score:  0.3111111111111111
0.0 0.800698804982403
Score:  0.7111111111111111
0.0 0.4101397315097063
Score:  0.3111111111111111
0.0 0.6484833025283671
Score:  0.7777777777777778
0.0 0.3639666576353995
Score:  0.3111111111111111
0.0 0.4603381228058666
Score:  0.4222222222222222
0.0 0.6450910244854432
Score:  0.4222222222222222
0.0 0.7194171795751392
Score:  0.28888888888888886
0.0 0.08298073065086864
Score:  0.35555555555555557
0.0 0.4386158349096493
Score:  0.28888888888888886
0.0 0.5328299445376992
Score:  0.4222222222222222
0.0 0.1441218106142551
Score:  0.28888888888888886
0.0 0.43450595061469166
Score:  0.28888888888888886
0.0 0.3207086367015013
Score:  0.4222222222222222
0.0 0.6914556029692845
Score:  0.28888888888888886
0.0 0.2626196306348151
Score:  0.28888888888888886
0.0 0.8492972921287296
Score:  0.28888888888888886
0.0 0.6123929231256264
Score:  0.35555555555555557
0.0 0.52629

Score:  0.35555555555555557
0.0 0.07277311581683166
Score:  0.28888888888888886
0.0 0.477472163870833
Score:  0.4222222222222222
0.0 0.8813812292274346
Score:  0.4222222222222222
0.0 0.383453824092317
Score:  0.4222222222222222
0.0 0.7810691696315691
Score:  0.4222222222222222
0.0 0.9889855331210129
Score:  0.4222222222222222
0.0 0.8901433301179931
Score:  0.7111111111111111
0.0 0.32194418025864324
Score:  0.3111111111111111
0.0 0.014809469848192536
Score:  0.7777777777777778
0.0 0.5276219800376516
Score:  0.3111111111111111
0.0 0.7439363966685507
Score:  0.4222222222222222
0.0 0.49221718251830726
Score:  0.28888888888888886
0.0 0.9689689466500897
Score:  0.7777777777777778
0.0 0.9328773688641413
Score:  0.7777777777777778
0.0 0.2783935385653272
Score:  0.28888888888888886
0.0 0.5071815241975839
Score:  0.28888888888888886
0.0 0.14692153109791384
Score:  0.7777777777777778
0.0 0.8784619723928797
Score:  0.7777777777777778
0.0 0.3833847012696495
Score:  0.8444444444444444
0.0 0.52136922

Score:  0.9333333333333333
0.0 0.0793033570885211
Score:  0.8222222222222222
0.0 0.4276955350969942
Score:  0.7777777777777778
0.0 0.003253239974601918
Score:  0.8222222222222222
0.0 0.23403849482086492
Score:  0.7777777777777778
0.0 0.2667202669441817
Score:  0.7777777777777778
0.0 0.7518493861389888
Score:  0.7777777777777778
0.0 0.25332042209698824
Score:  0.8222222222222222
0.0 0.1713555666694786
Score:  0.8222222222222222
0.0 0.9693101536559817
Score:  0.8222222222222222
0.0 0.7818439192647432
Score:  0.9333333333333333
0.0 0.5786649915893
Score:  0.8444444444444444
0.0 0.4982440012184278
Score:  0.7777777777777778
0.0 0.3239342264682412
Score:  0.7777777777777778
0.0 0.4779215592278392
Score:  0.28888888888888886
0.0 0.16325237336787768
Score:  0.7777777777777778
0.0 0.3206932426268818
Score:  0.28888888888888886
0.0 0.2776840992212801
Score:  0.7777777777777778
0.0 0.22578953431532311
Score:  0.7111111111111111
0.0 0.010199000183538809
Score:  0.28888888888888886
0.0 0.476776173

In [10]:
print('Best solution: ', solution)
print('Scored: ', score)

Best solution:  {'name': 'LogisticRegression', 'parameters': {'penalty': 'l1', 'C': 0.004832930238571752, 'solver': 'saga', 'n_jobs': -1}}
Scored:  1.0
